# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA
* Model: distilbert-base-uncased
* Evaluation approach: evaluate method with a HuggingFace Trainer
* Fine-tuning dataset: ctoraman/BilTweetNews-sentiment-analysis

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [2]:
! pip install -q "datasets==2.15.0" peft

In [8]:
from datasets import load_dataset

dataset = load_dataset("cajcodes/political-bias", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=23
)

splits = ["train", "test"]

# View the dataset characteristics
dataset["train"]

ModuleNotFoundError: No module named 'datasets'

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Let's use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["text"], truncation=True, padding='max_length', max_length=512), batched=True
    )

# Inspect the available columns in the dataset
tokenized_dataset["train"]

Map:   0%|          | 0/132 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 525
})

In [14]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=5,
    id2label={0: "highly conservative", 1: "slightly conservative", 2: "neutral", 3: "slightly liberal", 4: "highly liberal"},
    label2id={"highly conservative": 0, "slightly conservative": 1, "neutral": 2, "slightly liberal": 3, "highly liberal": 4},
)

# Unfreeze all the model parameters.
for param in model.parameters():
    param.requires_grad = True


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [7]:
from peft import get_peft_model, LoraConfig, TaskType

# Define LoRA configuration
config = LoraConfig(
    task_type=TaskType.SEQ_CLS,# Sequence Classification
    r=8,# Rank
    lora_alpha=32,# Alpha
    lora_dropout=0.1,# Dropout
    # Experiment with different target_modules
    target_modules=["q_lin", "v_lin"]  # or ["bert.encoder.layer.%i.attention.self.query" % i for i in range(6)]
)

# Enable input gradients
model.enable_input_require_grads()

# Apply LoRA to the model
model = get_peft_model(model, config)
print(model)

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): Linear(
                  in_features=768, out_features=768, bias=True
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=8, bias=Fals

In [8]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./results",
        # Set the learning rate
        learning_rate = 2e-3,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size = 4,
        per_device_eval_batch_size = 4,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.342244,0.409091
2,No log,1.158717,0.484848


TrainOutput(global_step=264, training_loss=1.4282428857051965, metrics={'train_runtime': 4586.6706, 'train_samples_per_second': 0.229, 'train_steps_per_second': 0.058, 'total_flos': 141491260108800.0, 'train_loss': 1.4282428857051965, 'epoch': 2.0})

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [10]:
trainer.evaluate()

{'eval_loss': 1.1587169170379639,
 'eval_accuracy': 0.48484848484848486,
 'eval_runtime': 205.6344,
 'eval_samples_per_second': 0.642,
 'eval_steps_per_second': 0.16,
 'epoch': 2.0}